# Climate Explorer Search Demo

This notebook demonstrates Sust Global's Climate Explorer "Search" functionality. This enables rapid retrieval of physical risk exposure at the individual asset/property level.

Use the "Notebook Configuration" section below to provide your Climate Explorer credentials, as well as a Google Maps API Key. Attendees of the Climate Data Studio will be given this information. To use your own Google Maps API key, please visit the [Google Maps API documentation](https://developers.google.com/maps/documentation/javascript/get-api-key).

The "Search Execution" section demonstrates address-based risk exposure retrieval. Configure `SEARCH_ADDRESS` with the value you wish to search for, then run the relevant cells in this notebook. The "Utility Functions" section contains the actual code used to interact with the Climate Explorer Search API.

## Notebook Configuration

In [1]:
import os

# Your Climate Explorer API key and project must be provided. Use the environment variables already
# wired up, or overwrite the following lines with your own values manually.
API_KEY = os.environ.get('SUST_API_KEY')
PROJECT = os.environ.get('SUST_PROJECT')

# A Google Maps API Key must be provided to support address geocoding. Provide this value via an
# environment variable, or override the value below.
GOOGLE_MAPS_API_KEY = os.environ.get('GOOGLE_MAPS_API_KEY')

## Utility Functions

In [2]:
from dataclasses import asdict, dataclass
from typing import List
import urllib.parse

import googlemaps
import pandas as pd
import requests


@dataclass
class Point:
    lat: float
    lng: float
    

def geocode(address: str) -> Point:
    cl = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
    res = cl.geocode(address)

    if len(res) != 1:
        raise Exception(f"geocoder returned {len(res)} results, expected 1")

    loc = res[0]['geometry']['location']
    return Point(**loc)


def get_physical_risk_exposure(point: Point) -> dict:
    qp = urllib.parse.urlencode({
        "lat": point.lat,
        "lng": point.lng,
    })
    
    headers = {
        "X-SustGlobal-APIKey": API_KEY, 
        "X-SustGlobal-Project": PROJECT
    }

    ep = f"https://explorer.sustglobal.io/api/search?{qp}"

    resp = requests.get(ep, headers=headers)
    if resp.status_code != 200:
        raise Exception(f"API request failed: code={resp.status_code} body={resp.text}")

    return resp.json()['features'][0]['properties']['physical_risk_exposure']

## Search Execution

The primary input to a search query is an address. This value is geocoded to set of coordinates using the Google Maps Geocoding API before submitting the query itself.

Note that the provided address MUST be located within the "lower 48" continguous United States.

In [3]:
# Define your search input below.
SEARCH_ADDRESS = "595 Pacific Ave, San Francisco, CA"

In [4]:
pt = geocode(SEARCH_ADDRESS)
print(f"Successfully resolved address: search=\"{SEARCH_ADDRESS}\" geom={pt}")

Successfully resolved address: search="595 Pacific Ave, San Francisco, CA" geom=Point(lat=37.7970119, lng=-122.4050949)


In [5]:
exp = get_physical_risk_exposure(pt)
print("Successfully retrieved risk summary data")

Successfully retrieved risk summary data


## Search Results

Below you will find the results of your search query. Find more information about individual hazards in the [Sust Global Climate Data Guide](https://developers.sustglobal.com/dataguide.html).

### Local Results

The local physical risk exposure summary below represents exposure at the location of interest within a ~10km radius.

In [6]:
pd.DataFrame(exp['local']['30yr']).sort_values(by=['hazard'])

,hazard,score,label
0,cyclones,0.000000,LOW
1,drought,0.200400,LOW
5,floods,0.000000,LOW
3,heatwaves,17.333334,LOW
4,sea_level_rise,0.143112,LOW
2,wildfire,0.000000,LOW


### Background Results

The background physical risk exposure summary below represents exposure at the location of interest within a ~30km radius.

In [7]:
pd.DataFrame(exp['background']['30yr']).sort_values(by=['hazard'])

,hazard,score,label
0,cyclones,0.000000,LOW
1,drought,0.239762,LOW
5,floods,0.000000,LOW
3,heatwaves,17.367512,LOW
4,sea_level_rise,0.113159,LOW
2,wildfire,0.000000,LOW
